In [ ]:
!pip install torchmetrics omegaconf pycolmap

In [3]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics
# check pytorch and pytorch.vision versions
import torch
import torchvision
from glob import glob
import numpy as np
import cv2

print("PyTorch version : ", torch.__version__)
print("\nTorchVisoion Version : ", torchvision.__version__)

# Load the model avaliable in hubconf
calibrator = torch.hub.load('AlanSavio25/DeepSingleImageCalibration', 'calibrator')

PyTorch version :  2.2.2+cu121

TorchVisoion Version :  0.17.2+cu121


In [4]:
# Define the image set for calibration
images = glob('../../images/virtual/set_5/*.jpg')

Using cache found in /home/mateuspincho/.cache/torch/hub/AlanSavio25_DeepSingleImageCalibration_master


In [5]:
focal_length = []
for image in images:

  results = calibrator.calibrate_from_path(image_path = image)
  focal_length.append(results['focal_length_pixels'])

print(focal_length)

[328.3190002441406, 464.32843017578125, 355.81463623046875, 386.5780334472656, 386.5780334472656]


In [6]:
f = np.median(focal_length)
resolution = (480,480)
o_x, o_y = resolution[0]/2, resolution[1]/2

camera_matrix = np.array([[  f,   0, o_x],
                          [  0,   f, o_y],
                          [  0,   0,   1]])
patternSize = (7,7)
rvecs = []
tvecs = []
errors = []

def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize=(7,7), squareSize = 30)

In [12]:
for image in images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
    _, rvec, tvec = cv2.solvePnP(boardPoints, corners, camera_matrix, 0)
    rvecs.append(rvec)
    tvecs.append(tvecs)

    image_points_deep, _ = cv2.projectPoints(boardPoints, rvec, tvec, camera_matrix, 0)

    error = cv2.norm(corners[0], image_points_deep[0], normType= cv2.NORM_L2) / len(image_points_deep[0])
    errors.append(error)

    print(f"Error of Image {image}:", error, "px")

print("Median error: ", np.median(errors))

Error of Image ../../images/virtual/set_5/image4.jpg: 0.6833765230146407 px


Error of Image ../../images/virtual/set_5/image1.jpg: 0.5338386620861659 px
Error of Image ../../images/virtual/set_5/image2.jpg: 0.3141629986431349 px
Error of Image ../../images/virtual/set_5/image3.jpg: 1.0260362425319527 px
Error of Image ../../images/virtual/set_5/image5.jpg: 0.21228562853366933 px
Median error:  0.5338386620861659
